# Delete useless jpgs found in the images folder 

In [ ]:
import os

u_files = [useless_file 
           for useless_file in os.listdir('/path/to/images/folder')
           if '._' in useless_file]

for u_file in u_files:
    os.remove(os.path.join('/path/to/images/folder', u_file))

# Dataset creation

Create the folders tree divided by training | validation | test and the resepctive supports

In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Paths
data_path = "C:/Users/leoac/Downloads/archive/images/images"
processed_prado = "C:/Users/leoac/OneDrive - Università degli Studi di Milano/Unimi/Subjects/2nd_year/02_quarter/Algorithms/project/processed_prado.feather"
base_dir = "C:/Users/leoac/Downloads/archive/dataset_3_classes"

# Load the .feather file
df = pd.read_feather(processed_prado)

# Define train and validation directories
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Create directories for each support type
support_types = df['support'].unique()
for support in support_types:
    os.makedirs(os.path.join(train_dir, support), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, support), exist_ok=True)

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['support'], random_state=42)

# Function to move files to the respective directories
def move_files(file_names, src_dir, dest_dir, support_type):
    for file_name in file_names:
        src_path = os.path.join(src_dir, file_name)
        dest_path = os.path.join(dest_dir, support_type, file_name)
        try:
            shutil.copy(src_path, dest_path)
        except:
            print(file_name)

# Move training files
for support in support_types:
    train_files = train_df[train_df['support'] == support]['image_file_name'].tolist()
    move_files(train_files, data_path, train_dir, support)

# Move validation files
for support in support_types:
    val_files = val_df[val_df['support'] == support]['image_file_name'].tolist()
    move_files(val_files, data_path, validation_dir, support)

print("Dataset organized successfully.")


4230ce8a-ff28-41a7-b5c6-932b86b1a62c.jpg
28212471-a0d5-483e-80a9-b4ad009bf25b.jpg
e7573626-3a12-4acd-b7eb-1db4a6d9aa42.jpg
86d75c9b-af1c-4e52-86fe-9ee9f7987ae8.jpg
f91fbc09-e595-48fe-b8af-e3636a52acfd.jpg
Dataset organized successfully.


In [2]:
import os
import shutil

dataset_dir = r'C:\Users\leoac\Downloads\archive\dataset_3_classes'
test_dir = os.path.join(dataset_dir, 'test')

# Create the test folder
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Function to move images from a specific category
def move_to_test(support):
    src_subdir = os.path.join(dataset_dir, 'validation', support)
    dst_subdir = os.path.join(test_dir, support)
    
    # Ensure the destination subdir exists
    if not os.path.exists(dst_subdir):
        os.makedirs(dst_subdir)
    
    # Move 10 images from the source subdir to the destination subdir
    for filename in os.listdir(src_subdir)[:10]:
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Adjust the extensions as needed
            src_file = os.path.join(src_subdir, filename)
            dst_file = os.path.join(dst_subdir, filename)
            shutil.move(src_file, dst_file)
            print(f'Moved {filename} from {support}')

# Call the function for each support
support_types = ['Papel', 'Tabla', 'Lienzo']
for support in support_types:
    move_to_test(support)

Moved 006f3780-ec36-41be-9273-27827d94ece8.jpg from Papel
Moved 01cfe574-afd5-499a-b8e4-481b9a2e21a9.jpg from Papel
Moved 04edc638-f488-4186-9c79-553025e823df.jpg from Papel
Moved 056c975b-9b22-418e-8055-68c5782dc5f3.jpg from Papel
Moved 05a68c55-cdba-480f-9edb-4aa2b9d965e7.jpg from Papel
Moved 06916e41-571a-4e14-a9a1-6cb7131efdc6.jpg from Papel
Moved 0b938333-1ab7-46e3-89c9-156b34530d4a.jpg from Papel
Moved 0d88f80a-6be3-43eb-95bb-5e5d2858a1e8.jpg from Papel
Moved 0e8a1808-e386-4b5f-b869-3ae5209b0e59.jpg from Papel
Moved 0f24cf11-4371-4e32-9f3e-df50ffd682b6.jpg from Papel
Moved 03a76a95-80e6-4f28-bdf6-5d4a5136c60b.jpg from Tabla
Moved 0561a40c-cca1-4357-a01f-b62801f1183a.jpg from Tabla
Moved 05d0ac32-067d-4234-9ff2-3c3f4e2c1b01.jpg from Tabla
Moved 08277e8d-bfd2-474e-a4eb-8c443948c74e.jpg from Tabla
Moved 0863779a-cff6-4780-a3cd-ce182188ad4f.jpg from Tabla
Moved 0b3aee1a-08c6-492a-ad4e-1ab327a5d933.jpg from Tabla
Moved 0d3e28cc-c62c-4231-aa43-b3a50de349e4.jpg from Tabla
Moved 0d6d199d

Delete the rows with missing images

In [2]:
values_to_drop = [
    '4230ce8a-ff28-41a7-b5c6-932b86b1a62c.jpg',
    '28212471-a0d5-483e-80a9-b4ad009bf25b.jpg',
    'e7573626-3a12-4acd-b7eb-1db4a6d9aa42.jpg',
    '86d75c9b-af1c-4e52-86fe-9ee9f7987ae8.jpg',
    'f91fbc09-e595-48fe-b8af-e3636a52acfd.jpg'
    ]
df = df.loc[~df['image_file_name'].isin(values_to_drop)]
df = df.reset_index(drop=True)
df.to_feather('processed_prado.feather')

# Load on GCP bucket

In [1]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import storage
from tqdm import tqdm

In [2]:
import os

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\leoac\OneDrive - Università degli Studi di Milano\Unimi\Subjects\2nd_year\02_quarter\Algorithms\project\prado-amd-project-f65f003b7777.json'

In [3]:
def upload_file(bucket_name, local_path, remote_path):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(remote_path)
    blob.upload_from_filename(local_path)
    print(f'File {local_path} uploaded to {remote_path}.')

def upload_files_to_gcs(bucket_name, source_folder):
    files_to_upload = []

    # Walk through the directory to collect files
    for root, dirs, files in tqdm(os.walk(source_folder)):
        for file in tqdm(files):
            local_path = os.path.join(root, file)
            remote_path = os.path.relpath(local_path, source_folder).replace("\\", "/")
            files_to_upload.append((bucket_name, local_path, remote_path))

    # Use ThreadPoolExecutor to upload files in parallel
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(upload_file, *args) for args in files_to_upload]
        for future in as_completed(futures):
            future.result()  # This will raise an exception if the upload failed

In [6]:
# def upload_file(bucket_name, local_path, remote_path):
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)
#     blob = bucket.blob(remote_path)
#     blob.upload_from_filename(local_path)
#     print(f'File {local_path} uploaded to {remote_path}.')

# def upload_files_to_gcs(bucket_name, source_folder):
#     files_to_upload = []

#     # Walk through the directory to collect files
#     for root, dirs, files in tqdm(os.walk(source_folder)):
#         for file in files:
#             local_path = os.path.join(root, file)
#             remote_path = os.path.relpath(local_path, source_folder).replace("\\", "/")
#             files_to_upload.append((bucket_name, local_path, remote_path))

#     # Iterate over files and upload them sequentially
#     for args in tqdm(files_to_upload):
#         upload_file(*args)

In [4]:
bucket_name = 'prado-images-bucket'
source_folder = r"C:\Users\leoac\Downloads\archive\dataset_3_classes"
# for root, dirs, files in os.walk(source_folder):
#     print(root, dirs, files)
#     for file in files:
#         local_path = os.path.join(root, file)
#         print(os.path.relpath(local_path, source_folder))
#         break
upload_files_to_gcs(bucket_name, source_folder)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 583/583 [00:00<00:00, 16446.15it/s]
0it [00:00, ?it/s]
100%|██████████| 137/137 [00:00<00:00, 18714.20it/s]
13it [00:00, 67.11it/s]


File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\0a5dea84-1563-48f5-ba2e-5eb3c7f4e18e.jpg uploaded to test/Lienzo/0a5dea84-1563-48f5-ba2e-5eb3c7f4e18e.jpg.
File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\00c8739f-cc77-44f2-8739-fa3c98ef6120.jpg uploaded to test/Lienzo/00c8739f-cc77-44f2-8739-fa3c98ef6120.jpg.
File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\0b490990-3666-4ac5-871d-cb36f0a551c5.jpg uploaded to test/Lienzo/0b490990-3666-4ac5-871d-cb36f0a551c5.jpg.
File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\050d4150-8d36-4535-9bf5-05f1dbbfbafb.jpg uploaded to test/Lienzo/050d4150-8d36-4535-9bf5-05f1dbbfbafb.jpg.
File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\0b60050b-0fbf-476b-9e31-56f6710f6cde.jpg uploaded to test/Lienzo/0b60050b-0fbf-476b-9e31-56f6710f6cde.jpg.
File C:\Users\leoac\Downloads\archive\dataset_3_classes\test\Lienzo\094fdc81-1ec1-4cc2-a1bb-051aa5e0af1b.jpg uploaded to test